<a href="https://colab.research.google.com/github/higebobo/my-colab/blob/main/pypnf_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pypfによるポイントアンドフィギュア算出

[pviglucci/pypf](https://github.com/pviglucci/pypf)

## インストールセクション

In [ ]:
!pip install requests_cache pypnf

     |████████████████████████████████| 47 kB 2.0 MB/s 
     |████████████████████████████████| 138 kB 9.8 MB/s 
     |████████████████████████████████| 127 kB 46.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## インポートセクション

In [ ]:
import datetime
import os

import pandas_datareader.data as pdr
from pypnf import PointFigureChart
from requests_cache import CachedSession
from termcolor import colored as cl

## 定数定義

### 対象のシンボル(銘柄・指数・通貨ペア)

In [ ]:
symbol_map = {
    'AMZN': {'name': 'Amazon.com Inc.', 'price_range': 50},
    'GOOGL': {'name': 'Alphabet Inc.', 'price_range': 50},
    '7203.T': {'name': 'トヨタ自動車'},
    '^N225': {'name': 'Nikkei225', 'price_range': 100},
    'JPY=X': {'name': 'USD/JPY', 'price_range': 0.1},
}

### 対象期間

In [ ]:
# 本日まで
end = datetime.date.today()
# 半年前まで
# end = datetime.date.today() - datetime.timedelta(weeks=26)

# 1年前から
start = end - datetime.timedelta(weeks=26*2)

### ポイントアンドフィギュアパラーメータ

In [ ]:
# 枠転換
reversal = 3

# 算出手法 (cl:終値, h/l:高値安値, hlc, ohlc, l/h)
method = 'cl'

# スケール  ('abs', 'log', 'cla')
scaling = 'abs'

# トレンドラインの描画
show_trendline = False

### ドライブマウントオプション

Google Driveのマイドライブ直下に時系列データをcsv形式で保存するオプション



In [ ]:
SAVE_GDRIVE = False

if SAVE_GDRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    PATH = '/content/drive/MyDrive'

## 関数

In [ ]:
def get_price_range_jp(price):
    """\
    国内株用の値幅算出
    [12\.P&F｜岡三オンライン証券](https://www.okasan-online.co.jp/tradeinfo/technical-analytics/articles/12_PandF/)
    """
    if price < 100:
        return 2.5
        # return 2
    elif price < 200:
        return 5
    elif price < 1000:
        return 10
    elif price < 5000:
        return 20
    elif price < 10000:
        return 1000
    elif price < 50000:
        return 200
    elif price < 100000:
        return 1000
    elif price < 500000:
        return 2000
    elif price < 1000000:
        return 10000
    elif price < 5000000:
        return 20000
    else:
        return 100000

In [ ]:
def get_price_range(price):
  """\
  米国株用の値幅算出
  [Introduction to Point & Figure Charts \[ChartSchool\]](https://school.stockcharts.com/doku.php?id=chart_analysis:pnf_charts:pnf_basics)
  """
  if price < 0.25:
    return 0.0625
  elif price < 1:
    return 0.125
  elif price < 5:
    return 0.25
  elif price < 20:
    return 0.50
  elif price < 100:
    return 1.00
  elif price < 200:
    return 2.00
  elif price < 500:
    return 4.00
  elif price < 1000:
    return 5.00
  elif price < 25000:
    return 50.00
  else:
    return 500.00


## 処理

requests_cacheの設定

In [ ]:
expire_after = datetime.timedelta(days=1) # 1日キャッシュ
session = CachedSession(cache_name='cache', backend='sqlite', 
                        expire_after=expire_after) 
session.headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Accept': 'application/json;charset=utf-8'
}

算出および描画

In [ ]:
for symbol, value in symbol_map.items():
    name = symbol_map[symbol]['name']
    
    # 時系列データの取得
    df = pdr.DataReader(symbol, 'yahoo', start, end, session=session)
    # print(df)
    if SAVE_GDRIVE:
        df.to_csv(os.path.join(PATH, f'{symbol}.csv'))

    # 値幅の設定
    price = df.iloc[-1]['Close']
    boxsize = symbol_map[symbol].get('price_range')
    if not boxsize:
        if symbol[:-2] == '.T':
            boxsize = get_price_range_jp(price)
        else:
            boxsize = get_price_range(price)

    # 描画
    ts = df.to_dict('list')
    pnf = PointFigureChart(ts=ts, method=method, reversal=reversal, boxsize=boxsize, scaling=scaling) #, title=cl(symbol, attrs=['bold']))
    if show_trendline:
        pnf.get_trendlines()

    print(cl(f'{name} ({symbol})', attrs=['bold']))
    print('\n')
    print(pnf)
    print('\n\n')

Amazon.com Inc. (AMZN)


Point & Figure (abs|cl) 50 x 3
----  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  ----
3700  X  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  3700
3650  X  O  .  .  X  .  .  .  .  .  .  .  .  .  .  .  3650
3600  X  O  .  .  X  O  .  .  .  .  .  .  .  .  .  .  3600
3550  X  O  .  .  X  O  .  .  .  .  .  .  .  .  .  .  3550
3500  X  O  X  .  X  O  .  .  .  .  .  .  .  .  .  .  3500
3450  X  O  X  O  X  O  .  .  .  .  .  .  .  .  .  .  3450
3400  X  O  X  O  X  O  .  .  .  .  .  .  .  .  .  .  3400
3350  X  O  X  O  X  O  .  .  .  .  .  .  X  .  .  .  3350
3300  X  O  X  O  X  O  .  .  .  .  .  .  X  O  .  .  3300
3250  X  O  X  O  X  O  .  .  .  .  .  .  X  O  .  .  3250
3200  X  O  .  O  .  O  .  .  X  .  .  .  X  O  .  .  3200
3150  .  .  .  .  .  O  .  .  X  O  .  .  X  O  .  .  3150
3100  .  .  .  .  .  O  .  .  X  O  .  .  X  O  .  .  3100
3050  .  .  .  .  .  O  .  .  X  O  X  .  X  O  .  .  3050
3000  .  .  .  .  .  O  X  .  X  O  X  O  X  O  .  .  3000
